## VADER Sentiment Analysis : VADER (Valence Aware Dictionary and sEntiment Reasoner) 

is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. VADER uses a combination of A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled according to their semantic orientation as either positive or negative. VADER not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.

In [1]:
# JAVA_HOME refers to jdk/bin directory. It is used by a java based application.
import os
jv = os.environ.get('JAVA_HOME', None)
jv

'/usr/lib/jvm/java-8-openjdk-amd64'

In [2]:
import findspark

findspark.init()

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = \
'--packages com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.0  pyspark-shell'
#  '--packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [4]:
!export PYTHONPATH="/opt/anaconda/envs/pyspark_env/pkgs/spark-nlp-3.0.0-py38_0/lib/python3.8/site-packages/sparknlp/:$PYTHONPATH"

In [5]:
!export PYTHONPATH="/opt/anaconda/envs/pyspark_env/pkgs/spark-nlp-3.0.0-py38_0/lib/python3.8/site-packages/:$PYTHONPATH"

In [6]:
!echo $PYTHONPATH

In [7]:
import sys, glob, os
sys.path.extend(glob.glob(os.path.join(os.path.expanduser("~"), ".ivy2/jars/*.jar")))

In [8]:
sys.path

['/opt/spark/python',
 '/opt/spark/python/lib/py4j-0.10.9-src.zip',
 '/home/hadoopuser',
 '/opt/anaconda/envs/pyspark_env/lib/python37.zip',
 '/opt/anaconda/envs/pyspark_env/lib/python3.7',
 '/opt/anaconda/envs/pyspark_env/lib/python3.7/lib-dynload',
 '',
 '/opt/anaconda/envs/pyspark_env/lib/python3.7/site-packages',
 '/opt/anaconda/envs/pyspark_env/lib/python3.7/site-packages/IPython/extensions',
 '/home/hadoopuser/.ipython',
 '/home/hadoopuser/.ivy2/jars/net.sf.trove4j_trove4j-3.0.3.jar',
 '/home/hadoopuser/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.11-2.4.3.jar',
 '/home/hadoopuser/.ivy2/jars/joda-time_joda-time-2.9.5.jar',
 '/home/hadoopuser/.ivy2/jars/org.joda_joda-convert-1.8.1.jar',
 '/home/hadoopuser/.ivy2/jars/org.mongodb_mongo-java-driver-3.12.5.jar',
 '/home/hadoopuser/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.603.jar',
 '/home/hadoopuser/.ivy2/jars/org.slf4j_slf4j-api-1.7.21.jar',
 '/home/hadoopuser/.ivy2/jars/org.rocksdb_rocksdbjni-6.5.3.jar',
 '/home/had

In [9]:
# If you want to modify the path to packages from within Python, you can do:

import sys
# sys.path.append('/where/module/lives/')
sys.path.append('/opt/anaconda/envs/pyspark_env/pkgs/spark-nlp-3.0.0-py38_0/lib/python3.8/site-packages/')

In [10]:
os.environ["TFHUB_CACHE_DIR"] = '/tmp/tfhub'

In [11]:
# Spark NLP
# https://www.gigaspaces.com/blog/natural-language-processing-examples/

import sparknlp.annotator

In [12]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
import json

# SPARK
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Spark NLP
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

https://www.gigaspaces.com/blog/natural-language-processing-examples/

In [13]:
spark=SparkSession.builder \
    .appName("Spark NLP_vs_VADER Sentiment Analysis") \
    .master("local[*]") \
    .config("spark.driver.memory","7G") \
    .config("spark.driver.maxResultSize", "5524M") \
    .config("spark.kryoserializer.buffer.max", "800M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.0") \
    .getOrCreate()

In [14]:
# from pyspark.sql import SparkSession

# spark  = SparkSession.builder \
#                       .master("local") \
#                       .appName("VADER Sentiment Analysis") \
#                       .getOrCreate()

In [15]:
# sc.stop()

In [14]:
spark

In [15]:
sc = spark.sparkContext

In [16]:
sc

<SparkContext master=local[*] appName=Spark NLP_vs_VADER Sentiment Analysis>

In [19]:
# spark.stop()

In [17]:
#MODEL_NAME='sentimentdl_use_imdb'
MODEL_NAME='sentimentdl_use_twitter'

In [18]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
# use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
use = UniversalSentenceEncoder.load("/home/hadoopuser/nlp/tfhub/")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


# sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
#     .setInputCols(["sentence_embeddings"])\
#     .setOutputCol("sentiment")

# nlpPipeline = Pipeline(
#       stages = [
#           documentAssembler,
#           use,
#           sentimentdl
#       ])

In [19]:
sparknlp.version()

'3.0.0'

In [20]:
sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [24]:
# empty_df = spark.createDataFrame([['']]).toDF("text")

# pipelineModel = nlpPipeline.fit(empty_df)

# # df = spark.createDataFrame(pd.DataFrame({"text":text_list}))

# result = pipelineModel.transform(df)

NameError: name 'df' is not defined

In [21]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("index", IntegerType()),
    StructField("text", StringType())
])

spark_tweetsDF = spark.read.csv('tweets.csv',header=True, schema = schema).dropna()

In [22]:
spark_tweetsDF.show(2,truncate=False)

+-----+-------------------------------------------------------------------------------------------------------+
|index|text                                                                                                   |
+-----+-------------------------------------------------------------------------------------------------------+
|0    |97% of the funding for the development of the Oxford/AstraZeneca Covid-19 vaccine came from the public.|
|1    |Would there be more uptake on AstraZeneca if there was some catchy jingle?                             |
+-----+-------------------------------------------------------------------------------------------------------+
only showing top 2 rows



### 6. Run the pipeline


In [23]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

# df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(spark_tweetsDF)

In [24]:
type(result)

pyspark.sql.dataframe.DataFrame

In [25]:
result.printSchema()

root
 |-- index: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |   

In [26]:
result.select('text', 'sentiment').toPandas()

,text,sentiment
0,97% of the funding for the development of the ...,"[(category, 0, 102, negative, {'sentence': '0'..."
1,Would there be more uptake on AstraZeneca if t...,"[(category, 0, 73, positive, {'sentence': '0',..."
2,ICYMI - The head of #Denmark's medicines agenc...,"[(category, 0, 146, negative, {'sentence': '0'..."
3,"""It will be a lot fun to work with @AstraZenec...","[(category, 0, 243, positive, {'sentence': '0'..."
4,I was vaccinated at the EXPO Centre Monday nig...,"[(category, 0, 158, positive, {'sentence': '0'..."
5,"Hmmmm. Not meaning to be conspiratorial, but I...","[(category, 0, 69, positive, {'sentence': '0',..."
6,Day #466 of the #pandemic: #covid 8x more li...,"[(category, 0, 97, negative, {'sentence': '0',..."
7,Danish Drug Director’s Collapse During AstraZe...,"[(category, 0, 88, neutral, {'sentence': '0', ..."
8,"""History's verdict on the decision of Western ...","[(category, 0, 191, negative, {'sentence': '0'..."
9,It sounds like Alberta is having trouble using...,"[(category, 0, 101, negative, {'sentence': '0'..."


In [27]:
from pyspark.sql.functions import col
# result1.select('text', col('sentiment')[0]).toPandas().style.set_properties(subset=['sentiment'], **{'width': '300px'})
# result1.select('text', col('sentiment')[0]['result'], col('sentiment')[0]['metadata']).show(truncate = False)

result_f = result.select('text', 'sentiment').withColumn('result_nlp', col('sentiment')[0]['result']).withColumn('negative_nlp', col('sentiment')[0]['metadata']['negative']) \
        .withColumn('positive_nlp', col('sentiment')[0]['metadata']['positive']) \
        .drop('sentiment')
# result1_f.style.set_properties(subset=['text'], **{'width': '300px'}).style.
result_f.toPandas().style.set_properties(subset=['text'], **{'width': '300px'})

,text,result_nlp,negative_nlp,positive_nlp
0,97% of the funding for the development of the Oxford/AstraZeneca Covid-19 vaccine came from the public.,negative,0.99518126,0.0048186826
1,Would there be more uptake on AstraZeneca if there was some catchy jingle?,positive,2.1342139E-38,1.0
2,"ICYMI - The head of #Denmark's medicines agency, Tanja Erichsen, collapsed during the press conference announcing the #AstraZeneca discontinuation.",negative,1.0,0.0
3,"""It will be a lot fun to work with @AstraZeneca addressing questions related to """"organelle adaptation"""" and drug resistance in acute leukemia! Big Thanks! Also congrats to @glytsou and all the #AifantisLab @nyugrossman https://t.co/NukgV0TW3w""",positive,0.0,1.0
4,"I was vaccinated at the EXPO Centre Monday night. Big thanks to all the staff who made the process so easy and efficient. When it is your turn, take the shot.",positive,6.565297E-30,1.0
5,"Hmmmm. Not meaning to be conspiratorial, but I'm no convinced likes 🤔",positive,3.567761E-5,0.99996436
6,Day #466 of the #pandemic: #covid 8x more likely to cause blood clots than #astrazenecavaccine :,negative,1.0,2.2729053E-10
7,Danish Drug Director’s Collapse During AstraZeneca Announcement.. https://t.co/p7SkZGt7H8,neutral,0.48423877,0.5157612
8,"""History's verdict on the decision of Western technocrats to stop vaccinating with AstraZeneca and Johnson & Johnson will be devastating. """"It’s sending vaccine confidence into a crater.""""",negative,1.0,1.9363352E-30
9,It sounds like Alberta is having trouble using its AstraZeneca. Lower the minimum age; Gen X can help!,negative,1.0,3.728038E-33


In [8]:
# sentiment = udf(lambda x: TextBlob(x).sentiment[0])
# spark.udf.register('sentiment', sentiment)
# tweets = tweets.withColumn('sentiment',sentiment('text').cast('double')

In [28]:
import nltk
import csv
import os
# from nltk

In [29]:
# /home/hadoopuser/nltk_data/sentiment/vader_lexicon.zip
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hadoopuser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## UDFs

### Create a DataFrame

In [30]:
data = [['study is going on as usual'], ['Goodmorning to everyone'], ['I am very sad today.'], ['Geeks For Geeks is the best portal for \
             the computer science engineering students.']]
columns = ['text']

data = [[' '.join(text[0].split())] for text in data]
print(data)

# df = spark.createDataFrame(rdd).toDF(*columns)
df = spark.createDataFrame(data=data,schema=columns)
df.show(truncate=False)

[['study is going on as usual'], ['Goodmorning to everyone'], ['I am very sad today.'], ['Geeks For Geeks is the best portal for the computer science engineering students.']]
+---------------------------------------------------------------------------------+
|text                                                                             |
+---------------------------------------------------------------------------------+
|study is going on as usual                                                       |
|Goodmorning to everyone                                                          |
|I am very sad today.                                                             |
|Geeks For Geeks is the best portal for the computer science engineering students.|
+---------------------------------------------------------------------------------+



In [31]:
df2 = spark.createDataFrame([
    ['Alabama',],
    ['Texas',],
    ['Antioquia',]
]).toDF('state')

df2.show(truncate=False)

+---------+
|state    |
+---------+
|Alabama  |
|Texas    |
|Antioquia|
+---------+



### Create and register a udf

In [32]:
import sys
from pyspark.sql.functions import udf 
# DoubleType, FloatType, ByteType, IntegerType, LongType, ShortType, ArrayType, StructField, StructType, Row
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pyspark.sql.types as Types
from nltk.sentiment.vader import SentimentIntensityAnalyzer


def sentiment_scores(sentance: str) -> dict :
    
    
    # Create a SentimentIntensityAnalyzer object.
    sid = SentimentIntensityAnalyzer()
    
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    r = sid.polarity_scores(sentance);
    return r


# You can optionally set the return type of your UDF. The default return type is StringType.
# udffactorial_p = udf(factorial_p, LongType())
sentiment_scoresUDF = udf(sentiment_scores, Types.MapType(Types.StringType(), Types.DoubleType()))



In [ ]:
# result 	negative 	positive
# result_f

In [33]:

from pyspark.sql.functions import col,sqrt,log,reverse
result_f1 = result_f.withColumn("sentiment", sentiment_scoresUDF(result_f.text))
# t.show()
result_f1.toPandas().style.set_properties(subset=['text'], **{'width': '300px'})

,text,result_nlp,negative_nlp,positive_nlp,sentiment
0,97% of the funding for the development of the Oxford/AstraZeneca Covid-19 vaccine came from the public.,negative,0.99518126,0.0048186826,"{'neg': 0.0, 'pos': 0.0, 'compound': 0.0, 'neu': 1.0}"
1,Would there be more uptake on AstraZeneca if there was some catchy jingle?,positive,2.1342139E-38,1.0,"{'neg': 0.0, 'pos': 0.0, 'compound': 0.0, 'neu': 1.0}"
2,"ICYMI - The head of #Denmark's medicines agency, Tanja Erichsen, collapsed during the press conference announcing the #AstraZeneca discontinuation.",negative,1.0,0.0,"{'neg': 0.11, 'pos': 0.0, 'compound': -0.2732, 'neu': 0.89}"
3,"""It will be a lot fun to work with @AstraZeneca addressing questions related to """"organelle adaptation"""" and drug resistance in acute leukemia! Big Thanks! Also congrats to @glytsou and all the #AifantisLab @nyugrossman https://t.co/NukgV0TW3w""",positive,0.0,1.0,"{'neg': 0.0, 'pos': 0.253, 'compound': 0.8802, 'neu': 0.747}"
4,"I was vaccinated at the EXPO Centre Monday night. Big thanks to all the staff who made the process so easy and efficient. When it is your turn, take the shot.",positive,6.565297E-30,1.0,"{'neg': 0.0, 'pos': 0.264, 'compound': 0.8659, 'neu': 0.736}"
5,"Hmmmm. Not meaning to be conspiratorial, but I'm no convinced likes 🤔",positive,3.567761E-5,0.99996436,"{'neg': 0.155, 'pos': 0.402, 'compound': 0.6652, 'neu': 0.443}"
6,Day #466 of the #pandemic: #covid 8x more likely to cause blood clots than #astrazenecavaccine :,negative,1.0,2.2729053E-10,"{'neg': 0.0, 'pos': 0.0, 'compound': 0.0, 'neu': 1.0}"
7,Danish Drug Director’s Collapse During AstraZeneca Announcement.. https://t.co/p7SkZGt7H8,neutral,0.48423877,0.5157612,"{'neg': 0.314, 'pos': 0.0, 'compound': -0.4939, 'neu': 0.686}"
8,"""History's verdict on the decision of Western technocrats to stop vaccinating with AstraZeneca and Johnson & Johnson will be devastating. """"It’s sending vaccine confidence into a crater.""""",negative,1.0,1.9363352E-30,"{'neg': 0.195, 'pos': 0.147, 'compound': -0.3818, 'neu': 0.659}"
9,It sounds like Alberta is having trouble using its AstraZeneca. Lower the minimum age; Gen X can help!,negative,1.0,3.728038E-33,"{'neg': 0.209, 'pos': 0.235, 'compound': 0.1511, 'neu': 0.556}"


In [34]:
result_f1.printSchema()

root
 |-- text: string (nullable = true)
 |-- result_nlp: string (nullable = true)
 |-- negative_nlp: string (nullable = true)
 |-- positive_nlp: string (nullable = true)
 |-- sentiment: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)



In [35]:
result_f1 = result_f1.withColumn('negative_nltk', col('sentiment')['neg']) \
                        .withColumn('positive_nltk', col('sentiment')['pos']) \
                        .withColumn('neutral_nltk', col('sentiment')['neu']) \
                        .withColumn('compound_nltk', col('sentiment')['compound']) \
                        .drop('sentiment')

result_f1.toPandas().style.set_properties(subset=['text'], **{'width': '300px'})

,text,result_nlp,negative_nlp,positive_nlp,negative_nltk,positive_nltk,neutral_nltk,compound_nltk
0,97% of the funding for the development of the Oxford/AstraZeneca Covid-19 vaccine came from the public.,negative,0.99518126,0.0048186826,0.000000,0.000000,1.000000,0.000000
1,Would there be more uptake on AstraZeneca if there was some catchy jingle?,positive,2.1342139E-38,1.0,0.000000,0.000000,1.000000,0.000000
2,"ICYMI - The head of #Denmark's medicines agency, Tanja Erichsen, collapsed during the press conference announcing the #AstraZeneca discontinuation.",negative,1.0,0.0,0.110000,0.000000,0.890000,-0.273200
3,"""It will be a lot fun to work with @AstraZeneca addressing questions related to """"organelle adaptation"""" and drug resistance in acute leukemia! Big Thanks! Also congrats to @glytsou and all the #AifantisLab @nyugrossman https://t.co/NukgV0TW3w""",positive,0.0,1.0,0.000000,0.253000,0.747000,0.880200
4,"I was vaccinated at the EXPO Centre Monday night. Big thanks to all the staff who made the process so easy and efficient. When it is your turn, take the shot.",positive,6.565297E-30,1.0,0.000000,0.264000,0.736000,0.865900
5,"Hmmmm. Not meaning to be conspiratorial, but I'm no convinced likes 🤔",positive,3.567761E-5,0.99996436,0.155000,0.402000,0.443000,0.665200
6,Day #466 of the #pandemic: #covid 8x more likely to cause blood clots than #astrazenecavaccine :,negative,1.0,2.2729053E-10,0.000000,0.000000,1.000000,0.000000
7,Danish Drug Director’s Collapse During AstraZeneca Announcement.. https://t.co/p7SkZGt7H8,neutral,0.48423877,0.5157612,0.314000,0.000000,0.686000,-0.493900
8,"""History's verdict on the decision of Western technocrats to stop vaccinating with AstraZeneca and Johnson & Johnson will be devastating. """"It’s sending vaccine confidence into a crater.""""",negative,1.0,1.9363352E-30,0.195000,0.147000,0.659000,-0.381800
9,It sounds like Alberta is having trouble using its AstraZeneca. Lower the minimum age; Gen X can help!,negative,1.0,3.728038E-33,0.209000,0.235000,0.556000,0.151100


In [38]:
result_f1.toPandas().to_excel(r'tweets_sentiment_analysis_results.xlsx', index = False,  header=True)
# encoding='utf8',
# VADER: https://github.com/cjhutto/vaderSentiment
# https://www.youtube.com/watch?v=4yjr3gC7OOQ
# https://www.got-it.ai/solutions/excel-chat/excel-tutorial/conditional-formatting/conditionally-format-a-cell

In [ ]:
 
    print(f"text = {text}, r = {r}")
    #                 positive_avg = (positive_avg + r['pos'])/cnt;
    #                 negative_avg = (negative_avg + r['neg'])/cnt;
    #                 neutral_avg = (neutral_avg + r['neu'])/cnt;
                    positive_avg = (positive_avg + r['pos'])
                    negative_avg = (negative_avg + r['neg'])
                    neutral_avg = (neutral_avg + r['neu'])

                print('*'*80)
                print(f'cnt = {cnt}')
                print(f'numRun = {i}')
                print(db_tweets)

        tweets_sentiments = {'positive_avg' : r['pos'], 'negative_avg' : r['neg'], 'neutral_avg' : r['neu']}
        return tweets_sentiments

### About the Scoring

    The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

    It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

        positive sentiment: compound score >= 0.05
        neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
        negative sentiment: compound score <= -0.05

    **NOTE**: The compound score is the one most commonly used for sentiment analysis by most researchers, including the authors.

    The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1... or close to it with float operation). These are the most useful metrics if you want to analyze the context & presentation of how sentiment is conveyed or embedded in rhetoric for a given sentence. For example, different writing styles may embed strongly positive or negative sentiment within varying proportions of neutral text -- i.e., some writing styles may reflect a penchant for strongly flavored rhetoric, whereas other styles may use a great deal of neutral text while still conveying a similar overall (compound) sentiment. As another example: researchers analyzing information presentation in journalistic or editorical news might desire to establish whether the proportions of text (associated with a topic or named entity, for example) are balanced with similar amounts of positively and negatively framed text versus being "biased" towards one polarity or the other for the topic/entity.
    
        IMPORTANTLY: these proportions represent the "raw categorization" of each lexical item (e.g., words, emoticons/emojis, or initialisms) into positve, negative, or neutral classes; they do not account for the VADER rule-based enhancements such as word-order sensitivity for sentiment-laden multi-word phrases, degree modifiers, word-shape amplifiers, punctuation amplifiers, negation polarity switches, or contrastive conjunction sensitivity.


In [43]:
# Create a SentimentIntensityAnalyzer object.
sid = SentimentIntensityAnalyzer()

# polarity_scores method of SentimentIntensityAnalyzer
# oject gives a sentiment dictionary.
# which contains pos, neg, neu, and compound scores.
r = sid.polarity_scores('Goodmorning to everyone I am sad');
print(r);

r = sid.polarity_scores('I am very sad today.');
print(r);


{'neg': 0.437, 'neu': 0.563, 'pos': 0.0, 'compound': -0.4767}
{'neg': 0.531, 'neu': 0.469, 'pos': 0.0, 'compound': -0.5256}


[Create DataFrame From Python Objects in pyspark](ivan-georgiev-19530.medium.com/create-dataframe-from-python-objects-in-pyspark-bd8e191b9ebd)

## Create a pyspark dataframe from a Row factory

In [43]:
# Create row factory user_row

from pyspark.sql import Row

user_row = Row("dob", "age", "is_fan")
lst1 = ['1990-05-03', '1994-09-23']
lst2 =[29,25]
lst3 = [True, False]
data = [user_row(*line) for line in zip(lst1, lst2, lst3)]

user_df = spark.createDataFrame(data)

In [44]:
user_df.printSchema()

root
 |-- dob: string (nullable = true)
 |-- age: long (nullable = true)
 |-- is_fan: boolean (nullable = true)



In [45]:
user_df.show()

+----------+---+------+
|       dob|age|is_fan|
+----------+---+------+
|1990-05-03| 29|  true|
|1994-09-23| 25| false|
+----------+---+------+



## Create a pyspark DataFframe from a named tuple

In [46]:
from collections import namedtuple

user_row = namedtuple("user_row", ("dob", "age", "is_fan"))
user_row.__new__.__defaults__ = (None, None, None)


# data = [
# user_row('1990-05-03', 29, True),
# user_row('1994-09-23', 25)
# ] 

lst1 = ['1990-05-03', '1994-09-23']
lst2 =[29,25]
lst3 = [True, False]
data = [user_row(*line) for line in zip(lst1, lst2, lst3)]

user_df = spark.createDataFrame(data, ['dob', 'age', 'is_fan'])
user_df.show()

+----------+---+------+
|       dob|age|is_fan|
+----------+---+------+
|1990-05-03| 29|  true|
|1994-09-23| 25| false|
+----------+---+------+



## Create pyspark DataFrame Specifying Schema as datatype String

With this method the schema is specified as string. The string uses the same format as the
string returned by the schema.simpleString() method. The struct and brackets
can be omitted.

Following schema strings are interpreted equally:



In [47]:
"struct<dob:string, age:int, is_fan: boolean>"
"dob:string, age:int, is_fan: boolean"

# data = [
# ('1990-05-03', 29, True),
# ('1994-09-23', 25, False)
# ] 

lst1 = ['1990-05-03', '1994-09-23']
lst2 =[29,25]
lst3 = [True, False]
data = [user_row(*line) for line in zip(lst1, lst2, lst3)]

user_schema = "dob:string, age:int, is_fan: boolean"
user_df = spark.createDataFrame(data, user_schema)
user_df.show()

+----------+---+------+
|       dob|age|is_fan|
+----------+---+------+
|1990-05-03| 29|  true|
|1994-09-23| 25| false|
+----------+---+------+



In [48]:
user_df.printSchema()

root
 |-- dob: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_fan: boolean (nullable = true)



In [49]:
"struct<dob:string, age:int, is_fan: boolean>"
"dob:string, age:int, is_fan: boolean"

# data = [
# ('1990-05-03', 29, True),
# ('1994-09-23', 25, False)
# ] 

lst1 = ['1990-05-03', '1994-09-23']
lst2 =[29,25]
lst3 = [True, False]
data = [user_row(*line) for line in zip(lst1, lst2, lst3)]

user_schema = "struct<dob:string, age:int, is_fan: boolean>"
user_df = spark.createDataFrame(data, user_schema)
user_df.show()

+----------+---+------+
|       dob|age|is_fan|
+----------+---+------+
|1990-05-03| 29|  true|
|1994-09-23| 25| false|
+----------+---+------+



## Create Schema from JSON String

First we need to parse the JSON string into python dictionary and than we can use
StructType.fromJSON to create StructType object.

In [54]:
import json
import pyspark.sql.types as st
schema_json_str = """
{
    "type": "struct",
    "fields": [
        {
            "name": "dob",
            "type": "string",
            "nullable": true,
            "metadata": {}
        },
        {
            "name": "age",
            "type": "integer",
            "nullable": true,
            "metadata": {}
        },
        {
            "name": "is_fan",
            "type": "boolean",
            "nullable": true,
            "metadata": {}
        }
    ]
} 
"""

In [55]:
# Parse JSON string into python dictionary
schema_dict = json.loads(schema_json_str)
# Create StructType from python dictionary
schema = st.StructType.fromJson(schema_dict)

# data = [
#     ('1990-05-03', 29, True),
#     ('1994-09-23', 25, False)
# ] 

lst1 = ['1990-05-03', '1994-09-23']
lst2 =[29,25]
lst3 = [True, False]
data = [user_row(*line) for line in zip(lst1, lst2, lst3)]

user_df = spark.createDataFrame(data, schema)
user_df.show()

+----------+---+------+
|       dob|age|is_fan|
+----------+---+------+
|1990-05-03| 29|  true|
|1994-09-23| 25| false|
+----------+---+------+



In [56]:
import pyspark.sql.functions as F
def remove_non_word_characters(col):
    return F.regexp_replace(col, "[^\\w\\s]+", "")

In [59]:
def test_remove_non_word_characters(spark):
    data = [
        ("jo&&se", "jose"),
        ("**li**", "li"),
        ("#::luisa", "luisa"),
        (None, None)
    ]
    df = spark.createDataFrame(data, ["name", "expected_name"])\
        .withColumn("clean_name", remove_non_word_characters(F.col("name")))
#     assert_column_equality(df, "clean_name", "expected_name")
    return df

In [60]:
m_df = test_remove_non_word_characters(spark)

In [61]:
m_df

DataFrame[name: string, expected_name: string, clean_name: string]

In [62]:
m_df.show()

+--------+-------------+----------+
|    name|expected_name|clean_name|
+--------+-------------+----------+
|  jo&&se|         jose|      jose|
|  **li**|           li|        li|
|#::luisa|        luisa|     luisa|
|    null|         null|      null|
+--------+-------------+----------+



In [65]:
wordsDF = spark.createDataFrame([('cat',), ('elephant',), ('rat',), ('rat',), ('cat', )], ['word'])
wordsDF.show()
print (type(wordsDF))
wordsDF.printSchema()


+--------+
|    word|
+--------+
|     cat|
|elephant|
|     rat|
|     rat|
|     cat|
+--------+

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- word: string (nullable = true)



In [66]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.sql.functions import lit, concat

pluralDF = wordsDF.select(concat(wordsDF.word,lit('s')).alias('word'))
pluralDF.show()

+---------+
|     word|
+---------+
|     cats|
|elephants|
|     rats|
|     rats|
|     cats|
+---------+



In [67]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.sql.functions import length
pluralLengthsDF = pluralDF.select(length(pluralDF.word))
pluralLengthsDF.show()

+------------+
|length(word)|
+------------+
|           4|
|           9|
|           4|
|           4|
|           4|
+------------+



In [69]:
pluralDF.first()

Row(word='cats')

In [70]:
pluralDF.first()[0]

'cats'

In [71]:
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from databricks_test_helper import Test
# TEST Using DataFrame functions to add an 's' (1b)
Test.assertEquals(pluralDF.first()[0], 'cats', 'incorrect result: you need to add an s')
Test.assertEquals(pluralDF.columns, ['word'], "there should be one column named 'word'")

ModuleNotFoundError: No module named 'databricks_test_helper'

In [68]:
# TEST Length of each word (1e)
from collections import Iterable
asSelf = lambda v: map(lambda r: r[0] if isinstance(r, Iterable) and len(r) == 1 else r, v)

Test.assertEquals(asSelf(pluralLengthsDF.collect()), [4, 9, 4, 4, 4],
                  'incorrect values for pluralLengths')

/opt/anaconda/envs/pyspark_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


NameError: name 'Test' is not defined